## 0. Dependencies

In [ ]:
# 5. Import the required modules

import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import os
import networkx as nx
import nx_arangodb as nxadb

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root", 
        password="RHr0KzkRUVlp61IisH8G", 
        name="DAC_devops_log",
        verify=True)

arango_graph = ArangoGraph(db)

G = nxadb.Graph(name="sprint1")

In [ ]:
print(arango_graph.schema)

In [ ]:
# from langchain_core.prompts import PromptTemplate

# AQL_GENERATION_TEMPLATE = """Task: Generate an ArangoDB Query Language (AQL) query from a User Input.

# You are an ArangoDB Query Language (AQL) expert responsible for translating a `User Input` into an ArangoDB Query Language (AQL) query.

# You are given an `ArangoDB Schema`. It is a JSON Object containing:
# 1. `Graph Schema`: Lists all Graphs within the ArangoDB Database Instance, along with their Edge Relationships.
# 2. `Collection Schema`: Lists all Collections within the ArangoDB Database Instance, along with their document/edge properties and a document/edge example.

# You may also be given a set of `AQL Query Examples` to help you create the `AQL Query`. If provided, the `AQL Query Examples` should be used as a reference, similar to how `ArangoDB Schema` should be used.

# Things you should do:
# - Think step by step.
# - Rely on `ArangoDB Schema` and `AQL Query Examples` (if provided) to generate the query.
# - Begin the `AQL Query` by the `WITH` AQL keyword to specify all of the ArangoDB Collections required.
# - Return the `AQL Query` wrapped in 3 backticks (```).
# - Use only the provided relationship types and properties in the `ArangoDB Schema` and any `AQL Query Examples` queries.
# - Only answer to requests related to generating an AQL Query.
# - If a request is unrelated to generating AQL Query, say that you cannot help the user.

# Things you should not do:
# - Do not use any properties/relationships that can't be inferred from the `ArangoDB Schema` or the `AQL Query Examples`. 
# - Do not include any text except the generated AQL Query.
# - Do not provide explanations or apologies in your responses.
# - Do not generate an AQL Query that removes or deletes any data.

# Under no circumstance should you generate an AQL Query that deletes any data whatsoever.

# ArangoDB Schema:
# {adb_schema}


# AQL Query: 
# """
# aql_prompt = PromptTemplate(
#     input_variables=["adb_schema"],
#     template=AQL_GENERATION_TEMPLATE,
# )

# aql_prompt.format(adb_schema=arango_graph.schema)


## Example 1: basic query

In [ ]:
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True,
        # aql_generation_prompt=aql_prompt

    )
    
    result = chain.invoke(query)

    return str(result["result"])

In [ ]:
text_to_aql_to_text("what is the most important task in my graph?")

In [ ]:
text_to_aql_to_text("what is the task that has the highest number of preceding tasks in my graph?")

In [ ]:
text_to_aql_to_text("Return all tasks and task_precede_task")

## 2: Example PageRank

In [ ]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.
        
        Your response:
    """).content

    return FINAL_RESULT, nx_to_text


In [ ]:
text_to_nx_algorithm_to_text("which task should be the most important and should be completed first if you are to use page rank?")

In [ ]:
# Create a directed graph for tasks
task_graph = nx.DiGraph()
task_graph.add_edges_from(task_precedence_edges)

# Step 2: Apply PageRank algorithm
pagerank_scores = nx.pagerank(task_graph)

## 3. Employee Importance

In [ ]:
QUERY_RESULT, CHAT_ANSWER = text_to_nx_algorithm_to_text("""
I need to understand the importance of my employees
                             
Help me create this output:
Employee | total pagerank of all the tasks this employee completed
                             
return, without any intepreting, the original output
""")

In [ ]:
import pandas as pd

# Create DataFrame
df = pd.DataFrame(QUERY_RESULT, columns=['EmpID', 'PageRankSum'])

# Display the DataFrame
print(df)

In [ ]:
QUERY_RESULT

## 4. Create schedule

In [ ]:
text_to_nx_algorithm_to_text("""
    Ignore start date and end date of any task
                             
    I will now need to set up a timeline to do all these tasks. I believe it can be done with respect to these conditions:
    - Using page rank, more important tasks should be done first
    - A task can only be done if all of its preceding tasks are done
                             
    Give me the order of all the tasks I need to do. Here is how I imagine you should do it:
    - For each task, calculate its pagerank
    - Calculate the order of tasks based on
    
""")